In [2]:
# %pip install Levenshtein
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

In [22]:
# sevMil = pd.read_csv("companies_sorted.csv")
ft = pd.read_csv("Forbes_top 2000_2022.csv")
db = pd.read_csv("sparql_2022-10-02_13-24-07Z.csv")

In [4]:
sevMil = sevMil[['name']]
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1M_" + sevMil['id'].astype(str)

In [26]:
ft = ft[['organizationName']]
ft['id'] =  range(len(ft))
ft['id'] = "ft_" + ft['id'].astype(str)

In [27]:
dbpedia = db[['name']]
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "db_" + dbpedia['id'].astype(str)

<ipython-input-27-49af2a1eed77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] =  range(len(db))
<ipython-input-27-49af2a1eed77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbpedia['id'] = "db_" + dbpedia['id'].astype(str)


In [28]:
new_df = pd.DataFrame(product(ft['organizationName'], dbpedia['name']), columns=["ft","dbpedia"])

new_df["LevScore"] = new_df.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

In [29]:
new_df2 = new_df[new_df['LevScore'] > 0.8]

In [30]:
confirmed_true = new_df2[new_df2['LevScore'] == 1]
edge_cases = new_df2[new_df2['LevScore'] != 1]

In [31]:
true_sample = confirmed_true.sample(200)
false_sample = edge_cases.sample(200)

In [33]:
gold_standard_ft_db = pd.concat([true_sample,false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]
gold_standard_ft_db['label'] = gold_standard_ft_db.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ft_db

,ft_id,db_id,LevScore,ft,dbpedia,label
0,ft_1443,db_738,1.000000,Kimco Realty,Kimco Realty,True
1,ft_1402,db_1440,1.000000,Mitsubishi Materials,Mitsubishi Materials,True
2,ft_838,db_1450,1.000000,TDK,TDK,True
3,ft_310,db_4133,1.000000,Macquarie Group,Macquarie Group,True
4,ft_807,db_2629,1.000000,Mapfre,Mapfre,True
...,...,...,...,...,...,...
401,ft_426,db_5217,0.875000,DNB Bank,DBS Bank,False
402,ft_1712,db_6106,0.812500,Sumec Corporation,SPX Corporation,False
403,ft_846,db_1633,0.823529,MTN Group,BT Group,False
404,ft_808,db_4429,0.812500,HCL Technologies,TOA Technologies,False
